# Drainage

We need to salute the lady of the lake

In [ ]:
#| default_exp water/basin

# Hydrology System Design Document

## Overview
Create a comprehensive drainage and watershed analysis system that visualizes how water flows across terrain, including rivers, lakes, and watershed boundaries.

## File Structure
```
hydrology.py  # New file for drainage/watershed analysis
├── Lake detection and management
├── Watershed computation
├── Drainage network tracing
└── Rendering utilities
```

## Core Concepts

### 1. Lakes
**Definition**: Land hexes that are local minima - water cannot flow downhill to escape.

**Detection**:
- A hex is a lake if `elevation > 0` AND all neighbors have `elevation >= current`
- Multi-hex lakes: Adjacent local minima should merge into single lake bodies
- Endorheic basins: Watersheds that terminate at lakes instead of ocean

**Lake Class**:
```python
@dataclass
class Lake:
    hexes: set[int]  # All hexes in this lake body
    surface_elevation: float  # Water surface level
    watershed_id: int  # Links to watershed that drains here
    overflow_point: int = None  # Lowest point on lake edge (for future overflow)
```

### 2. Watersheds
**Definition**: A group of hexes that all drain to the same terminal point (ocean outlet or lake).

**Types**:
- **Ocean watersheds**: Drain to ocean (most common)
- **Lake watersheds**: Drain to an interior lake (endorheic)

**Watershed Class**:
```python
@dataclass
class Watershed:
    terminal_hex: int  # Ocean hex or lake hex
    land_hexes: list[int]  # All hexes that drain here
    drainage_paths: dict[int, list[int]]  # source_hex -> path to terminal
    color: str  # Categorical color from seaborn palette
    is_lake_basin: bool = False
    lake: Lake = None  # If this is a lake watershed
```

### 3. Drainage Networks
**Definition**: The complete set of flow paths from every land hex to its terminal point.

**Properties**:
- Every land hex traces downhill until hitting ocean or lake
- Path stores intermediate hexes for river rendering
- Accumulates flow for width calculations

## Key Algorithms

### Algorithm 1: Find Lakes
```python
def find_lakes(terrain: Terrain) -> dict[int, Lake]:
    """
    Detect all lakes (local minima) and merge adjacent ones.
    
    Returns:
        {lake_id: Lake} - One entry per distinct lake body
    """
    # Step 1: Find all local minima
    local_minima = set()
    for i in land_hexes:
        lowest_neighbor = terrain.lowest_neighbor(i)
        if lowest_neighbor is None or terrain.elevations[lowest_neighbor] >= terrain.elevations[i]:
            local_minima.add(i)
    
    # Step 2: Merge adjacent minima using flood fill
    lakes = {}
    visited = set()
    lake_id = 0
    
    for seed in local_minima:
        if seed in visited:
            continue
        
        # Flood fill at this elevation
        lake_hexes = set()
        queue = [seed]
        elevation = terrain.elevations[seed]
        
        while queue:
            current = queue.pop(0)
            if current in visited:
                continue
            if terrain.elevations[current] != elevation:
                continue
            
            lake_hexes.add(current)
            visited.add(current)
            
            # Add neighbors at same elevation
            for neighbor in terrain.ring(current, 1):
                if neighbor in local_minima:
                    queue.append(neighbor)
        
        lakes[lake_id] = Lake(
            hexes=lake_hexes,
            surface_elevation=elevation,
            watershed_id=lake_id
        )
        lake_id += 1
    
    return lakes
```

### Algorithm 2: Compute Watersheds
```python
def compute_watersheds(terrain: Terrain) -> dict[int, Watershed]:
    """
    Trace drainage from every land hex to terminal point.
    Group by terminal to form watersheds.
    
    Returns:
        {watershed_id: Watershed}
    """
    lakes = find_lakes(terrain)
    lake_hex_to_id = {}  # Reverse lookup
    for lake_id, lake in lakes.items():
        for hex in lake.hexes:
            lake_hex_to_id[hex] = lake_id
    
    # Trace every land hex
    terminal_map = {}  # land_hex -> (terminal_hex, path)
    
    for i in land_hexes:
        if i in lake_hex_to_id:
            # This hex IS a lake - it's its own terminal
            terminal_map[i] = (i, [i])
            continue
        
        path = [i]
        current = i
        visited = {i}
        
        while True:
            # Check if we hit a lake
            if current in lake_hex_to_id:
                terminal_map[i] = (current, path)
                break
            
            lowest = terrain.lowest_neighbor(current)
            
            # Dead end or loop
            if lowest is None or lowest in visited:
                terminal_map[i] = (current, path)
                break
            
            # Hit ocean
            if terrain.elevations[lowest] <= 0:
                terminal_map[i] = (lowest, path)
                break
            
            path.append(lowest)
            visited.add(lowest)
            current = lowest
    
    # Group by terminal
    watersheds_by_terminal = {}
    for land_hex, (terminal, path) in terminal_map.items():
        if terminal not in watersheds_by_terminal:
            watersheds_by_terminal[terminal] = {
                'land_hexes': [],
                'paths': {}
            }
        watersheds_by_terminal[terminal]['land_hexes'].append(land_hex)
        watersheds_by_terminal[terminal]['paths'][land_hex] = path
    
    # Create Watershed objects with colors
    colors = get_watershed_colors(len(watersheds_by_terminal))
    watersheds = {}
    
    for watershed_id, (terminal, data) in enumerate(watersheds_by_terminal.items()):
        is_lake = terminal in lake_hex_to_id
        
        watersheds[watershed_id] = Watershed(
            terminal_hex=terminal,
            land_hexes=data['land_hexes'],
            drainage_paths=data['paths'],
            color=colors[watershed_id],
            is_lake_basin=is_lake,
            lake=lakes[lake_hex_to_id[terminal]] if is_lake else None
        )
    
    return watersheds
```

### Algorithm 3: River Width from Gradient
```python
def compute_gradient_width(terrain: Terrain, path: list[int], 
                          min_width: float, max_width: float) -> list[float]:
    """
    Calculate river width at each segment based on elevation gradient.
    Steep = thick, gentle = thin.
    """
    widths = []
    
    # Find max gradient across all terrain for normalization
    max_gradient = 0
    for i in range(len(path) - 1):
        current = path[i]
        next_hex = path[i + 1]
        
        elev_drop = abs(terrain.elevations[current] - terrain.elevations[next_hex])
        distance = terrain.hexGrid.radius * 2  # Approximate hex-to-hex distance
        gradient = elev_drop / distance
        max_gradient = max(max_gradient, gradient)
    
    # Calculate width for each segment
    for i in range(len(path) - 1):
        current = path[i]
        next_hex = path[i + 1]
        
        elev_drop = abs(terrain.elevations[current] - terrain.elevations[next_hex])
        distance = terrain.hexGrid.radius * 2
        gradient = elev_drop / distance
        
        # Normalize and scale
        normalized = gradient / max_gradient if max_gradient > 0 else 0
        width = min_width + normalized * (max_width - min_width)
        widths.append(width)
    
    return widths
```

## Rendering Modes

### Mode 1: "drainage"
Shows complete drainage network colored by watershed.

**Render steps**:
1. Compute watersheds
2. For each watershed:
   - Fill lake hexes (if present) with watershed color
   - Draw all drainage paths as rivers, colored by watershed
   - River width based on gradient

### Mode 2: "watersheds"
Shows watershed boundaries only (like topographic maps).

**Render steps**:
1. Compute watersheds
2. Find boundary hexes (neighbors belong to different watersheds)
3. Draw boundary lines between watersheds

### Mode 3: "flow_accumulation"
Heatmap showing how many upstream hexes drain through each point.

**Render steps**:
1. Compute watersheds
2. For each hex, count upstream contributors
3. Color by accumulation (blue = low, red = high)

## Configuration

```python
@dataclass
class HydroRenderConfig:
    mode: str = "drainage"  # "drainage", "watersheds", "flow_accumulation"
    
    # Rivers
    show_major_rivers_only: bool = False
    major_river_threshold: int = 5  # min path length
    width_mode: str = "gradient"  # "gradient", "flow", "constant"
    river_min_width: float = 1.0
    river_max_width: float = 6.0
    
    # Lakes
    show_lakes: bool = True
    darken_lakes: bool = True  # Make lake slightly darker than watershed color
    lake_darken_factor: float = 0.8  # Multiply RGB by this
    
    # Colors
    palette: str = "tab20"  # Seaborn palette name
    
    # Display
    show_legend: bool = True
```

## Integration with Existing Code

### StyleCSS Integration
Use `StyleCSS.seaborn()` to generate watershed colors:
```python
def get_watershed_colors(n_watersheds: int, palette: str = "tab20") -> list[str]:
    """Generate N distinct colors using seaborn palette."""
    styles = StyleCSS.seaborn(palette, levels=n_watersheds)
    return [style.properties['fill'] for style in styles]
```

### Terraform Integration
Add hydro rendering to `Terraform.render_climate()`:
```python
if config.hydro is not None:
    hydro_layer = self.render_hydro(config.hydro)
    ret += hydro_layer
```

## Future Enhancements

1. **Lake overflow**: When lakes fill, find lowest point on edge to create outlet
2. **Seasonal variation**: Adjust river width by precipitation season
3. **Confluence highlighting**: Mark where major tributaries join
4. **Watershed statistics**: Area, total flow, average elevation
5. **Interactive**: Click watershed to highlight its drainage basin

## Testing Strategy

1. **Synthetic terrains**: Create known patterns (single peak, valley, basin)
2. **Lake detection**: Verify multi-hex lakes merge correctly
3. **Watershed count**: Should match number of ocean outlets + interior lakes
4. **Flow conservation**: Every land hex should drain somewhere
5. **Visual inspection**: Compare with real-world drainage patterns (Maui, California)

## Prior Art

In [ ]:
#| export
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum
import heapq

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

#custom
import copy

In [ ]:
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))

In [ ]:
#| export
from HexMagic.styles import StyleCSS, SVGBuilder, SVGLayer, SVGPatternLoader, preview, app, StyleDemo, LayerAnimation
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, hexBackground, HexRegion, unique_windy_edge
from HexMagic.terrain import  TerraDemo, Terrain, GeoBounds, ClimatePreset
from HexMagic.terrainpatterns import TerrainPatterns, SVGMask

In [ ]:
#| export
from HexMagic.water.soil import SoilSystem, SoilType
from HexMagic.water.river import River, RiverDemo
from HexMagic.weather import TerraDemo
from HexMagic.water.watershed import Watershed


## Drainage

In [ ]:
#| export
class DrainageBasins:

    def __init__(self,terrain: Terrain,debug=False):
        self.terrain = terrain
        self.sheds = Watershed.compute_all(terrain,debug=debug)

In [ ]:
#| export
@patch
def select_shed(self: DrainageBasins, 
                                    min_flow: int = 10,
                                    max_rivers: int = 20,
                                    min_length: int = 5) -> List[Watershed]:
    """Select the most important rivers to display.
    
    Strategy:
    1. One main river per major watershed (largest basins)
    2. Rivers must have significant flow (min_flow)
    3. Rivers must be long enough to be visible (min_length)
    4. Prioritize by watershed size and flow
    
    Args:
        min_flow: Minimum accumulated flow to show
        max_rivers: Maximum number of rivers to return
        min_length: Minimum number of hexes in river path
    
    Returns:
        List of River objects suitable for display
    """
    candidates = []
    
    for watershed in self.sheds:
        river = watershed.tributary
        
        # Calculate river metrics
        flow_values = river._calculate_flow()
        max_flow = max(flow_values.values()) if flow_values else 0
        river_length = len(river.hexes)
        watershed_size = len(watershed.region.hexes)
        
        # Score = combination of flow, length, and watershed size
        score = max_flow * 0.5 + river_length * 0.3 + watershed_size * 0.2
        
        # Filter by minimum criteria
        if max_flow >= min_flow and river_length >= min_length:
            candidates.append((watershed, score, max_flow, river_length))
    
    # Sort by score descending
    candidates.sort(key=lambda x: x[1], reverse=True)
    
    # Return top N watershed
    return [watershed for watershed, _, _, _ in candidates[:max_rivers]]
    

In [ ]:
#| export
@patch
def get_major(self: DrainageBasins, top_n: int = 5) -> List[Watershed]:
    """Get the N largest rivers by flow."""
    return self.select_shed(
        min_flow=2,      # Lower threshold for small terrains
        max_rivers=top_n,
        min_length=3
    )

In [ ]:
#| export
@patch
def watershed_overlay(self:DrainageBasins):
    """Demo showing watersheds colored by drainage basin."""

    # Create overlay for watershed colors
    overlay = ""
    
    for i, watershed in enumerate(self.sheds):
        # Add this watershed's style to builder
        self.terrain.hexGrid.builder.add_style(watershed.style)
        
        # Draw all hexes in this watershed's region
        for hex_idx in watershed.region.hexes:
            hex_obj = self.terrain.hexGrid.hexes[hex_idx]
            overlay += f"\t{hex_obj.svg()}\n".replace(
                hex_obj.style.name, 
                watershed.style.name
            )
    
    # Add watershed overlay as new layer
    return overlay
    
@patch
def boundary_overlay(self:DrainageBasins,
                boundary_style = StyleCSS("watershed_boundary", 
                             fill="none", 
                             stroke="#000000", 
                             stroke_width=2,
                             opacity=0.8)):
    # Optionally draw watershed boundaries
    overlay = ""
    self.terrain.hexGrid.builder.add_style(boundary_style)
    
    for watershed in self.sheds:
        paths = watershed.region.trace_perimeter(style=boundary_style)
        for path in paths:
            overlay += path.svg()
    
    return overlay


In [ ]:
#| export
@patch
def build_fans_and_deltas(self: DrainageBasins,
                          top_n: int = 8,
                          build_fans: bool = True,
                          build_deltas: bool = True,
                          debug: bool = False):
    """Build alluvial fans and deltas for major watersheds.
    
    Returns dict with 'fans' and 'deltas' keys containing HexRegion lists.
    """
    major_sheds = self.get_major(top_n)
    
    all_fans = []
    all_deltas = []
    
    for watershed in major_sheds:
        if build_fans:
            fans = watershed.build_alluvial_fan(debug=debug)
            all_fans.extend(fans)
        
        if build_deltas and watershed.is_ocean:
            delta = watershed.build_delta(debug=debug)
            if delta.hexes:
                all_deltas.append(delta)
    
    return {
        'fans': all_fans,
        'deltas': all_deltas
    }

def test_fans_and_deltas(debug=True, carve=True):
    """Test alluvial fan and delta creation."""
    
    # 1. Create terrain
    terrain = TerraDemo().california_map()
    
    if carve:
        rivers = terrain.carve_to_ocean(num_lakes=5)
    
    terrain.compute_weather()
    
    # 2. Compute watersheds
    basins = DrainageBasins(terrain, debug=debug)
    print(f"\nFound {len(basins.sheds)} watersheds")
    
    # 3. Snapshot elevations before
    elev_before = terrain.elevations.copy()
    
    # 4. Build fans and deltas
    result = basins.build_fans_and_deltas(top_n=8, debug=debug)
    
    fans = result['fans']
    deltas = result['deltas']
    
    print(f"\n=== RESULTS ===")
    print(f"Fans created: {len(fans)}")
    print(f"Deltas created: {len(deltas)}")
    
    # 5. Check elevation changes
    elev_diff = terrain.elevations - elev_before
    modified_hexes = np.sum(elev_diff != 0)
    print(f"Hexes with elevation changes: {modified_hexes}")
    print(f"Max elevation increase: {elev_diff.max():.1f}m")
    
    # 6. Fan stats
    if fans:
        fan_sizes = [len(f.hexes) for f in fans]
        print(f"\nFan sizes: min={min(fan_sizes)}, max={max(fan_sizes)}, avg={sum(fan_sizes)/len(fan_sizes):.1f}")
    
    # 7. Delta stats
    if deltas:
        delta_sizes = [len(d.hexes) for d in deltas]
        print(f"Delta sizes: min={min(delta_sizes)}, max={max(delta_sizes)}, avg={sum(delta_sizes)/len(delta_sizes):.1f}")
    
    # 8. Render visualization
    #terrain.render_climate()
    
    # Draw fans in orange
    fan_style = StyleCSS("fan_overlay", fill="#ff8800", opacity=0.6, stroke="none")
    terrain.builder.add_style(fan_style)
    fan_svg = ""
    for fan in fans:
        fan_svg += fan.draw(style=fan_style, inset=0.1)
    
    # Draw deltas in cyan
    delta_style = StyleCSS("delta_overlay", fill="#00cccc", opacity=0.6, stroke="none")
    terrain.builder.add_style(delta_style)
    delta_svg = ""
    for delta in deltas:
        delta_svg += delta.draw(style=delta_style, inset=0.1)
    
    terrain.builder.adjust("fans", fan_svg)
    terrain.builder.adjust("deltas", delta_svg)
    
    return terrain, basins, result

# Run it
terrain, basins, result = test_fans_and_deltas(debug=True)
terrain.builder.show()


### Layers

In [ ]:
#| export
@patch
def gradient_overlay(self: DrainageBasins,
                     min_width: float = 0.5,
                     max_width: float = 4.0,
                     opacity: float = 0.6):
    """Create overlay showing flow direction with gradient-based line widths.
    
    Each line segment:
    - Connects a land hex to its downhill neighbor
    - Colored by watershed
    - Width scaled by elevation gradient (steeper = thicker)
    
    Args:
        min_width: Minimum line width (for flat areas)
        max_width: Maximum line width (for steep slopes)
        opacity: Line opacity (0-1)
    
    Returns:
        SVG string of line segments
    """
    terrain = self.terrain
    
    # First pass: calculate all gradients to find max for normalization
    gradients = []
    flow_data = []  # Store (hex_idx, neighbor_idx, gradient, watershed_idx)
    
    # Map each hex to its watershed
    hex_to_watershed = {}
    for i, watershed in enumerate(self.sheds):
        for hex_idx in watershed.region.hexes:
            hex_to_watershed[hex_idx] = i
    
    for hex_idx in hex_to_watershed.keys():
        # Find lowest neighbor (flow direction)
        lowest = terrain.lowest_neighbor(hex_idx)
        
        if lowest is None or terrain.elevations[lowest] <= 0:
            continue  # Skip if no downhill or flows to ocean
        
        # Calculate gradient
        elev_current = terrain.elevations[hex_idx]
        elev_next = terrain.elevations[lowest]
        elev_drop = elev_current - elev_next
        
        # Distance between hex centers
        hex_obj = terrain.hexGrid.hexes[hex_idx]
        neighbor_obj = terrain.hexGrid.hexes[lowest]
        distance = hex_obj.center.distance(neighbor_obj.center)
        
        gradient = elev_drop / distance if distance > 0 else 0
        gradients.append(gradient)
        
        watershed_idx = hex_to_watershed[hex_idx]
        flow_data.append((hex_idx, lowest, gradient, watershed_idx))
    
    # Find max gradient for normalization
    max_gradient = max(gradients) if gradients else 1.0
    
    # Second pass: generate SVG lines
    overlay = ""
    
    # Create styles for each watershed with opacity
    for i, watershed in enumerate(self.sheds):
        # Create a modified style with opacity for lines
        line_style = StyleCSS(
            f"{watershed.style.name}_flow",
            stroke=watershed.style.properties.get('fill', '#000000'),
            fill="none",
            opacity=opacity,
            stroke_linecap="round"
        )
        terrain.hexGrid.builder.add_style(line_style)
    
    # Generate line segments
    for hex_idx, neighbor_idx, gradient, watershed_idx in flow_data:
        # Normalize gradient and calculate width
        normalized = gradient / max_gradient if max_gradient > 0 else 0
        width = min_width + normalized * (max_width - min_width)
        
        # Get hex centers
        hex_obj = terrain.hexGrid.hexes[hex_idx]
        neighbor_obj = terrain.hexGrid.hexes[neighbor_idx]
        
        # Get watershed style
        watershed = self.sheds[watershed_idx]
        style_name = f"{watershed.style.name}_flow"
        
        # Create line segment
        overlay += (
            f'\t<line '
            f'x1="{hex_obj.center.x:.1f}" '
            f'y1="{hex_obj.center.y:.1f}" '
            f'x2="{neighbor_obj.center.x:.1f}" '
            f'y2="{neighbor_obj.center.y:.1f}" '
            f'class="{style_name}" '
            f'stroke-width="{width:.2f}" '
            f'/>\n'
        )
    
    return overlay


In [ ]:
#| export
@patch
def dotted_watershed_overlay(self: DrainageBasins, 
                              flow_levels: int = 5,
                              min_density: float = 0.25,  # NEW: minimum dot density
                              debug: bool = False) -> str:
    """Create dotted watershed overlay where dot density represents flow accumulation.
    
    Args:
        flow_levels: Number of dot density levels (default 5)
        min_density: Minimum density for low-flow areas (0.25 = 25%)
        debug: Print pattern generation info
    
    Returns:
        SVG string for the overlay
    """
    terrain = self.terrain
    
    # Ensure we have flow data
    if 'flow' not in terrain.fields:
        all_flows = {}
        for watershed in self.sheds:
            river_flow = watershed.tributary._calculate_flow()
            for hex_idx, flow_count in river_flow.items():
                all_flows[hex_idx] = all_flows.get(hex_idx, 0) + flow_count
        
        terrain.fields['flow'] = np.zeros(len(terrain.elevations))
        for hex_idx, flow in all_flows.items():
            terrain.fields['flow'][hex_idx] = flow
    
    flow_data = terrain.fields['flow']
    
    # Generate patterns for each watershed
    patternGen = TerrainPatterns(terrain)
    all_patterns = []
    watershed_to_patterns = {}
    
    for i, watershed in enumerate(self.sheds):
        color = watershed.style.properties.get('fill', '#cccccc')
        
        # Create flow_levels density patterns for this watershed color
        patterns = patternGen.ballDensity(
            levels=flow_levels + 3,
            fills=[color],
            prefix=f"watershed_{i}_flow"
        )[:flow_levels]
        
        if debug:
            print(f"Watershed {i}: generated {len(patterns)} patterns with color {color}")
        
        # Store pattern indices
        start_idx = len(all_patterns)
        watershed_to_patterns[i] = list(range(start_idx, start_idx + flow_levels))
        all_patterns.extend(patterns)
    
    # Build overlay
    grid = terrain.hexGrid
    overlay = ""
    used_patterns = set()
    
    # Map each hex to its watershed
    hex_to_watershed = {}
    for i, watershed in enumerate(self.sheds):
        for hex_idx in watershed.region.hexes:
            hex_to_watershed[hex_idx] = i
    
    # Find max flow for normalization
    watershed_hexes = set(hex_to_watershed.keys())
    max_flow = max((flow_data[h] for h in watershed_hexes if flow_data[h] > 0), default=1.0)
    
    if debug:
        print(f"Max flow in watersheds: {max_flow}")
        print(f"Min density: {min_density * 100}%")
    
    # Generate hex polygons
    for hex_idx in watershed_hexes:
        if hex_idx not in hex_to_watershed:
            continue
        
        watershed_idx = hex_to_watershed[hex_idx]
        flow = flow_data[hex_idx]
        
        # Normalize flow to [min_density, 1.0] range
        if flow > 0 and max_flow > 0:
            normalized_flow = (flow / max_flow)  # [0, 1]
            # Scale to [min_density, 1.0]
            scaled_flow = min_density + normalized_flow * (1.0 - min_density)
            flow_level = int(scaled_flow * (flow_levels - 1))
            flow_level = min(flow_level, flow_levels - 1)
        else:
            flow_level = 0  # Use minimum density pattern
        
        # Get pattern index
        pattern_indices = watershed_to_patterns[watershed_idx]
        pattern_idx = pattern_indices[flow_level]
        
        used_patterns.add(pattern_idx)
        
        # Generate polygon
        patName = all_patterns[pattern_idx].attributes['id']
        fill = f"url(#{patName})"
        
        hex_obj = grid.hexes[hex_idx]
        ret = "<polygon points=\""
        for point in hex_obj.vertices():
            ret += f"{point.x:.0f},{point.y:.0f} "
        ret += f"\" style=\"fill:{fill}\"/>"
        
        overlay += "\t" + ret + "\n"
    
    # Add used patterns to builder
    for pattern_idx in sorted(used_patterns):
        grid.builder.add_definition(all_patterns[pattern_idx])
    
    if debug:
        print(f"Used {len(used_patterns)} patterns out of {len(all_patterns)}")
    
    return overlay


In [ ]:
#| export
@patch
def draw_watersheds(self: DrainageBasins, top_n: int = 8, simplify_k: int = 3, max_width: float = None,lake_base_size: int = 3) -> str:

    watersheds = self.get_major(top_n)
    
    if max_width is None:
        max_width = self.terrain.hexGrid.radius * 2 / 3
    # Compute max flow across all selected rivers
    max_flow = 0
    for ws in watersheds:
        flows = ws.calculate_flow()
        if flows:
            max_flow = max(max_flow, max(flows.values()))
    
    # Draw each watershed, passing max_flow for absolute scaling
    svg = ""
    for shed in watersheds:
        #svg += shed.simplify(simplify_k).draw(max_flow=max_flow)
        svg += shed.draw(max_flow=max_flow,
            max_width=max_width,
            lake_base_size=lake_base_size
        )
    
    return svg


In [ ]:
@patch
def basin_demo(self:TerraDemo):
   terrain = self.california_map()
   terrain.compute_weather()
   terrain.colorMap()
   terrain.hexGrid.update()

   basins = DrainageBasins(terrain)
   terrain.builder.adjust("watersheds", basins.draw_watersheds())
   return terrain.builder.show()

In [ ]:
TerraDemo().basin_demo()

## Demos

In [ ]:
@patch
def simpleIsland(demo: TerraDemo, carve=False, build_deltas=False, debug=False):
    
    # 1. Create blank ocean world with tropical preset
    mySize = MapSize(500, 300)
    myBounds = MapRect(MapCord(0,0), mySize)
    
    t, plates = Terrain.fromSeeds(
        myBounds, radius=15, num_plates=10, 
        formation_type='ocean_distance',  # 'ridge', 'volcanic', 'rift', 'rolling'
    # Fine-tuning
    elevation_scale=1.0,
        oceanic_sides=['N'],
         
        age='young',  # Fewer subdivisions = clearer boundaries
        seed=42
    )
    t.colorMap()
    t.hexGrid.update()

    t.geo = GeoBounds(
        lat_min=20.57,   # Southern tip (near Makena)
        lat_max=21.03,   # Northern tip (near Kahakuloa)
        lon_min=-156.69, # Western tip (West Maui)
        lon_max=-155.97  # Eastern tip (Haleakalā/Hāna)
    )
    
    # Compute hex coordinates
    t._compute_hex_coordinates()
    
    # Maui-specific precipitation model
    # Trade winds from northeast at ~50-60 degrees
    t.climate = ClimatePreset(
    name='Maui',
    lat_range=(20.57, 21.03),  # Your actual CA bounds
    base_temp_range=(26, 28),
    wind_speed=8.0,        # Slightly stronger trade winds
    wind_dir=50.0,         
    precip_base=0.15,      # MUCH higher base moisture (tropical ocean)
    nm=0.008,              # Less stable (more convection)
    hw=2500.0,             # Higher moisture scale height
    cw=0.003,              # MUCH stronger orographic effect
    conv_time=1000.0,      # Faster conversion
    fall_time=1000.0 
    )
    t.compute_weather()

    terrain = t
    
    if carve:
        rivers = terrain.carve_to_ocean(num_lakes=5)
    
    # Build deltas from major watersheds
    if build_deltas:
        basins = DrainageBasins(terrain, debug=debug)
        major_watersheds = basins.get_major(top_n=5)
        
        for watershed in major_watersheds:
            if watershed.is_ocean:  # Only ocean-draining watersheds get deltas
               terrain.buildUp(watershed,rings=3,ele=7)

    return terrain


In [ ]:
@patch
def demoWatersheds(self: TerraDemo):
    """Demo showing watersheds colored by drainage basin."""
    
    # Create simplified island terrain
    terrain = self.simpleIsland()
    #terrain.hexGrid.update(layer_name="elevations")
    
    # Compute all watersheds
    basin = DrainageBasins(terrain)
    
    terrain.hexGrid.builder.adjust("watersheds", basin.dotted_watershed_overlay(min_density=0.5))
    terrain.hexGrid.builder.adjust("watershed_boundaries", basin.boundary_overlay())
    terrain.hexGrid.builder.adjust("borders",terrain.elevation_borders())

       # Add gradient flow lines
    gradient_overlay = basin.gradient_overlay(
        min_width=0.5,
        max_width=4.0,
        opacity=0.7
    )
    #terrain.hexGrid.builder.adjust("gradient_flow", gradient_overlay)


    river_style = StyleCSS(
        "nile",
        fill = "none",
        
        stroke= '#cad1d829',
        stroke_width=3,
        opacity= 0.7
    )
    
    terrain.hexGrid.builder.add_style(river_style)
    river_svg = basin.draw_watersheds(top_n=6 , simplify_k=2)

    terrain.hexGrid.builder.adjust("rivers", river_svg)

    # Add legend showing watershed count
    legend_text = f"{len(basin.sheds)} Drainage Basins"
    
    terrain.hexGrid.builder.add_centered_text(
        legend_text, 
        y_offset=-terrain.hexGrid.builder.height/2 + 30,
        class_name="watershed_legend"
    )
    
    legend_style = StyleCSS("watershed_legend",
                           fill="#333333",
                           font_size="18px",
                           font_weight="bold",
                           font_family="Arial, sans-serif")
    terrain.hexGrid.builder.add_style(legend_style)
    
    return terrain #.hexGrid.builder.show()


In [ ]:
TerraDemo().demoWatersheds().hexGrid.builder.show()

### Start

In [ ]:
waterTer = TerraDemo().demoWatersheds()
waterTer.compute_precipitation_sb()
#waterTer.hexGrid.builder.show()

array([1.37665247e+03, 3.14702821e+02, 8.76000000e+01, 8.76000000e+01,
       8.76000000e+01, 8.76000000e+01, 8.76000000e+01, 4.84687838e+02,
       9.11462387e+02, 1.13751318e+03, 1.42841226e+03, 1.76021698e+03,
       1.81932638e+03, 2.12279517e+03, 2.51621646e+03, 2.39238795e+03,
       2.53043117e+03, 2.95636861e+03, 3.33302972e+03, 3.41294331e+03,
       3.50986267e+03, 3.42977191e+03, 3.46937296e+03, 3.31092065e+03,
       3.42436101e+03, 3.64093243e+03, 3.52947518e+03, 3.75671891e+03,
       3.93353647e+03, 3.95212345e+03, 4.07710304e+03, 4.14773366e+03,
       3.59669716e+03, 1.01850440e+03, 8.76000000e+01, 8.76000000e+01,
       8.76000000e+01, 8.76000000e+01, 8.76000000e+01, 8.76000000e+01,
       8.76000000e+01, 3.07173141e+02, 6.26070141e+02, 1.21033254e+03,
       1.45224118e+03, 1.82833477e+03, 2.52380301e+03, 2.58992698e+03,
       2.52590281e+03, 2.73843733e+03, 3.19661096e+03, 3.41270824e+03,
       3.65389075e+03, 3.76265431e+03, 3.82203383e+03, 3.94669556e+03,
      

In [ ]:
@patch
def demoLakes(self: TerraDemo):
    """Demo showing watersheds colored by drainage basin."""
    
    # Create simplified island terrain
    terrain = self.simpleIsland()
    terrain.hexGrid.update(layer_name="elevations")
    
    # Compute all watersheds
    basin = DrainageBasins(terrain)

     
    
    terrain.hexGrid.builder.adjust("watersheds", basin.dotted_watershed_overlay(min_density=0.5))
    #terrain.hexGrid.builder.adjust("watershed_boundaries", basin.boundary_overlay())
    terrain.hexGrid.builder.adjust("borders",terrain.elevation_borders())

       # Add gradient flow lines
    gradient_overlay = basin.gradient_overlay(
        min_width=0.5,
        max_width=4.0,
        opacity=0.7
    )
    #terrain.hexGrid.builder.adjust("gradient_flow", gradient_overlay)


    river_style = StyleCSS(
        "nile",
        fill = "none",
        
        stroke= '#cad1d829',
        stroke_width=3,
        opacity= 0.7
    )
    
    terrain.hexGrid.builder.add_style(river_style)
    river_svg = ""
    #print(basin.terrain)
    rivers = basin.get_major(6)
    
    for river in rivers:
        small_river = river.simplify(2)
        #print(small_river.terrain, river.terrain)
        if river.terrain is not None:
            small_river.tributary.terrain = terrain
            river_svg += small_river.draw(lake_base_size=1, lake_log_scale=2.0)


    terrain.hexGrid.builder.adjust("rivers", river_svg)

    watersheds = basin.sheds
    
    # Add legend showing watershed count
    legend_text = f"{len(watersheds)} Drainage Basins"
    
    terrain.hexGrid.builder.add_centered_text(
        legend_text, 
        y_offset=-terrain.hexGrid.builder.height/2 + 30,
        class_name="watershed_legend"
    )
    
    legend_style = StyleCSS("watershed_legend",
                           fill="#333333",
                           font_size="18px",
                           font_weight="bold",
                           font_family="Arial, sans-serif")
    terrain.hexGrid.builder.add_style(legend_style)
    
    return terrain

In [ ]:
laketerr = TerraDemo().demoLakes()
laketerr.hexGrid.builder.show()

## Drawing